In [10]:
import pandas as pd

data = pd.read_excel("./OpArticles/OpArticles_ADUs.xlsx")

In [11]:
from sklearn.model_selection import train_test_split

X = data.drop('label', axis=1)
y = data.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [12]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

df_train.to_excel("./OpArticles/Augmented/train_before_trans.xlsx", index=False)
df_test.to_excel("./OpArticles/Augmented/test.xlsx", index=False)

In [18]:
from googletrans import Translator
import random
import time

trans = Translator()

def augment(token, n_aug):
    aug_tokens = set()
    languages = ["en", "es", "fr", "fi", "no", "nl", "da", "ru", "de", "ja", "zh-cn"]
    count = 0
    while len(aug_tokens) < n_aug and count < 10:
        lang = random.choice(languages)
        t_token = trans.translate(token, src="pt", dest=lang).text
        t_token = trans.translate(t_token, src=lang, dest="pt").text
        aug_tokens.add(t_token)
        count += 1
        time.sleep(1)
        
    # print(aug_tokens)
    return aug_tokens

In [19]:
augment(X_test.tokens.iloc[4], 5)

{'Diminuição da circulação de conteúdo jornalístico profissional nas redes sociais seguida de aumento na circulação de notícias falsas',
 'a diminuição da circulação de conteúdo jornalístico profissional nas redes sociais é acompanhada por um aumento na circulação de notícias falsas',
 'o declínio na circulação de conteúdo jornalístico profissional nas redes sociais é acompanhado por um aumento na circulação de notícias falsas',
 'o declínio na distribuição de conteúdo jornalístico profissional nas redes sociais é acompanhado pela disseminação de notícias falsas'}

In [20]:
label_distribution = df_train.label.value_counts().to_dict()
print(label_distribution)
max_label_count = max(label_distribution.items(), key=lambda x:x[1])[1]
print(max_label_count)

{'Value': 6481, 'Fact': 2930, 'Value(-)': 2320, 'Value(+)': 1129, 'Policy': 534}
6481


In [21]:
from math import ceil
import numpy as np

aug_df = pd.DataFrame()

for label, count in label_distribution.items():
    label_diff = max_label_count - count
    aug_factor = ceil(label_diff / count)
    
    print(f"WILL NEED {aug_factor} TRANSLATIONS FOR CLASS {label}")

    if label_diff > 0:
        label_df = df_train[df_train["label"] == label]
        aug_label_df = pd.DataFrame()
        og_label_df = pd.DataFrame()

        for token in label_df.tokens:
            tk_df = pd.DataFrame(augment(token, aug_factor), columns=["tokens"])
            tk_df["label"] = label
            
            aug_label_df = aug_label_df.append(tk_df)
            og_label_df = og_label_df.append({"label": label, "tokens": token}, ignore_index=True)
        
        aug_label_df = aug_label_df.take(np.random.permutation(len(aug_label_df))[:label_diff])
        aug_df = aug_df.append([og_label_df, aug_label_df])
    else:
        aug_df = aug_df.append(df_train[df_train["label"] == label][["tokens", "label"]])

display(aug_df)

WILL NEED 0 TRANSLATIONS FOR CLASS Value
WILL NEED 2 TRANSLATIONS FOR CLASS Fact


ReadTimeout: The read operation timed out

In [ ]:
aug_label_distribution = aug_df.label.value_counts().to_dict()
print(aug_label_distribution)


{'Value': 25, 'Fact': 25, 'Value(-)': 25, 'Value(+)': 21}
